<img src="http://i67.tinypic.com/2jcbwcw.png" align="left"></img><br><br><br><br>


## Breakout Lecture 8: Web scraping & crawling

**Author List**: Alexander Fred Ojala

**Original Sources**: https://www.crummy.com/software/BeautifulSoup/bs4/doc/

**License**: Feel free to do whatever you want to with this code

## Setup and load packages

In [1]:
# stretch Jupyter coding blocks to fit screen
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
# make it run on py2 and py3
from __future__ import division, print_function

In [3]:
import bs4 as bs # Beautiful Soup is a Python library for pulling data out of HTML and XML files.
import urllib.request # The urllib.request module defines functions and classes which help in opening URLs (mostly HTTP)

## Scrape the Data-X Syllabus site

In [110]:
# Always use requests or urllib to open a URL and read in the source code before
# using Beautifulsoup to extract the information you want.

source = urllib.request.urlopen('https://data-x.blog/syllabus-data-x/').read() # open the URL and read in the content as a string of HTML code
print(source) # this is the source code for the site, looks very unstructured right now

b'<!DOCTYPE html>\n<html lang="en">\n<head>\n<meta charset="UTF-8">\n<meta name="viewport" content="width=device-width">\n<link rel="profile" href="http://gmpg.org/xfn/11">\n<link rel="pingback" href="https://data-x.blog/xmlrpc.php">\n\n<title>Syllabus: Data-X &#8211; Data-X</title>\n\t\t<script src=\'https://r-login.wordpress.com/remote-login.php?action=js&amp;host=data-x.blog&amp;id=120928364&amp;t=1488948987&amp;back=https%3A%2F%2Fdata-x.blog%2Fsyllabus-data-x%2F\' type="text/javascript"></script>\n\t\t<script type="text/javascript">\n\t\t/* <![CDATA[ */\n\t\t\tif ( \'function\' === typeof WPRemoteLogin ) {\n\t\t\t\tdocument.cookie = "wordpress_test_cookie=test; path=/";\n\t\t\t\tif ( document.cookie.match( /(;|^)\\s*wordpress_test_cookie\\=/ ) ) {\n\t\t\t\t\tWPRemoteLogin();\n\t\t\t\t}\n\t\t\t}\n\t\t/* ]]> */\n\t\t</script>\n\t\t<link rel=\'dns-prefetch\' href=\'//s2.wp.com\' />\n<link rel=\'dns-prefetch\' href=\'//s0.wp.com\' />\n<link rel=\'dns-prefetch\' href=\'//datax911.wordpr

In [111]:
# Create your Beautifulsoup object

soup = bs.BeautifulSoup(source,features='lxml') # interact with this object, looks like source in browser.

# source should be a string of HTML code
# features = 'lxml' is the parser. Sometimes it might be better to use 'html.parser'

In [74]:
# look at the printed source above. Title of the website is <title>Syllabus: Data-X &#8211; Data-X</title>. 
# Let's try to parse that with beautifulsoup

print(soup.title)

<title>Syllabus: Data-X – Data-X</title>


In [112]:
# or

print(soup.find('title'))

<title>Syllabus: Data-X – Data-X</title>


In [113]:
print(soup.title.name) # name of the tag for soup.title (should be 'title' of course)

title


In [114]:
# .string on a Tag type object returns a NavigableString type object.

print(soup.title.string)

Syllabus: Data-X – Data-X


In [115]:
# On the other hand, .text gets all the child strings and return concatenated using the given separator. 
# Return type of .text is unicode object.

print(soup.title.text)

Syllabus: Data-X – Data-X


### Example when there is a difference in child strings

In [121]:
print(soup.tr)

<tr>
<td width="36"><strong>Lec #</strong></td>
<td width="131"><strong>Topic</strong></td>
<td width="72"><strong>Tools</strong></td>
<td width="108"><strong>Cookbook Examples</strong></td>
<td width="68"><strong>HW DUE</strong></td>
<td width="104"><strong>Lab<br/>
</strong></td>
</tr>


In [119]:
print(soup.tr.string) # returns None, because no string type object in between <tr>

None


In [122]:
print(soup.tr.text) # returns all child strings within the <td> tags as well


Lec #
Topic
Tools
Cookbook Examples
HW DUE
Lab




In [123]:
print(soup.p) #finds first p tag in the HTML code

<p>Instructor: Ikhlaq Sidhu<br/>
Department of Industrial Engineering &amp; Operations Research</p>


In [124]:
soup.find_all('p') #finds all p tags

[<p>Instructor: Ikhlaq Sidhu<br/>
 Department of Industrial Engineering &amp; Operations Research</p>,
 <p>Offered Spring 2017, 3 Units, Lecture and Lab:</p>,
 <p>Prerequisite: Interested students should have working knowledge of Python in advance of the class, and also should have completed a fundamental probability or statistics course.</p>,
 <p>New Location: Now at 3108 Etcheverry Hall, Time: 5:10 pm-7:59<br/>
 Not Barrows 60</p>,
 <p>Teaching Team:</p>,
 <p>Office Hours:<br/>
 Tue 1:30-2:30pm at Etcheverry 4176 (Breakout Room B)</p>,
 <p>This course surveys a variety of key of concepts that are useful for designing and building applications that process data signals.  The course also introduces modern open source, computer programming tools and libraries that can be used to implement these applications.  These concepts include filtering, prediction, classification, decision-making, Markov chains, LTI systems, spectral analysis, and frameworks for learning from data.    After review

In [125]:
soup.find_all('p').text # cannot use method on the full ResultSet, place in a loop instead

AttributeError: 'ResultSet' object has no attribute 'text'

In [231]:

# remove tag
# can remove with regex, but here we use beautifulsoups built in functionality
for paragraph in soup.find_all('p'):
    print(paragraph.text) # returns regular unicode .string returns navigable text
    # child tag removes if we have span etc
    
# as we can see the table is not nicely structured

Instructor: Ikhlaq Sidhu
Department of Industrial Engineering & Operations Research
Offered Spring 2017, 3 Units, Lecture and Lab:
Prerequisite: Interested students should have working knowledge of Python in advance of the class, and also should have completed a fundamental probability or statistics course.
New Location: Now at 3108 Etcheverry Hall, Time: 5:10 pm-7:59
Not Barrows 60
Teaching Team:
Office Hours:
Tue 1:30-2:30pm at Etcheverry 4176 (Breakout Room B)
This course surveys a variety of key of concepts that are useful for designing and building applications that process data signals.  The course also introduces modern open source, computer programming tools and libraries that can be used to implement these applications.  These concepts include filtering, prediction, classification, decision-making, Markov chains, LTI systems, spectral analysis, and frameworks for learning from data.    After reviewing each concept, we explore implementing it within sample applications using Py

In [128]:
print(soup.get_text()) # Get all child strings, concatenated, will print all javascript, wordpress coding text in a nice fashion







Syllabus: Data-X – Data-X


		/* <![CDATA[ */
			if ( 'function' === typeof WPRemoteLogin ) {
				document.cookie = "wordpress_test_cookie=test; path=/";
				if ( document.cookie.match( /(;|^)\s*wordpress_test_cookie\=/ ) ) {
					WPRemoteLogin();
				}
			}
		/* ]]> */
		








		/* <![CDATA[ */
		function addLoadEvent(func) {
			var oldonload = window.onload;
			if (typeof window.onload != 'function') {
				window.onload = func;
			} else {
				window.onload = function () {
					oldonload();
					func();
				}
			}
		}
		/* ]]> */
	

			window._wpemojiSettings = {"baseUrl":"https:\/\/s0.wp.com\/wp-content\/mu-plugins\/wpcom-smileys\/twemoji\/2\/72x72\/","ext":".png","svgUrl":"https:\/\/s0.wp.com\/wp-content\/mu-plugins\/wpcom-smileys\/twemoji\/2\/svg\/","svgExt":".svg","source":{"concatemoji":"https:\/\/s0.wp.com\/wp-includes\/js\/wp-emoji-release.min.js?m=1488818651h&ver=4.7.3"}};
			!function(a,b,c){function d(a){var b,c,d,e,f=String.fromCharCode;if(!k||!k.fillText)return!1

In [244]:
# Find first url

first_url = soup.find('a')
print('first_url:', first_url,'\n')

print('Type:',type(first_url))
print('Text: ',first_url.text)
print('Attributes:',first_url.attrs)

first_url: <a class="header-image-link" href="https://data-x.blog/" rel="home" title="Data-X">
<img alt="" height="154" src="https://datax911.files.wordpress.com/2016/12/cropped-banner_matrix1.jpg" width="912"/>
</a> 

Type: <class 'bs4.element.Tag'>
Text:  


Attributes: {'href': 'https://data-x.blog/', 'title': 'Data-X', 'rel': ['home'], 'class': ['header-image-link']}


In [248]:
for url in soup.find_all('a'):
    print(url.get('href')) # get method get specific tag

https://data-x.blog/
https://data-x.blog/
#content
/
https://data-x.blog/
https://data-x.blog/syllabus-data-x/
https://data-x.blog/breakouts/
https://data-x.blog/contact/
https://piazza.com/class/iy6i5458rva2ud
https://wordpress.com/?ref=footer_blog
#


In [249]:
# imagine we only want to extract http links and write them to a file called data-x-urls.txt on a separate line 

# find all url links at the page
links = list()
for url in soup.find_all('a'):
    link = url.get('href')
    if 'http' in link:
        print(link)
        links.append(link+'\n')

# create/open a txt file with write, will overwrite if there is a file called data-x-urls
with open('data-x-urls.txt', 'w') as file: 
    file.writelines(links) #

https://data-x.blog/
https://data-x.blog/
https://data-x.blog/
https://data-x.blog/syllabus-data-x/
https://data-x.blog/breakouts/
https://data-x.blog/contact/
https://piazza.com/class/iy6i5458rva2ud
https://wordpress.com/?ref=footer_blog


In [250]:
# Only find URL's in the navigation bar (tag nav)
nav=soup.nav
nav

<nav class="main-navigation" id="site-navigation" role="navigation">
<h1 class="menu-toggle">Menu</h1>
<div class="screen-reader-text skip-link"><a href="#content" title="Skip to content">Skip to content</a></div>
<div class="menu-primary-container"><ul class="menu" id="menu-primary"><li class="menu-item menu-item-type-custom menu-item-object-custom menu-item-8" id="menu-item-8"><a href="/">Home</a></li>
<li class="menu-item menu-item-type-post_type menu-item-object-page menu-item-home menu-item-9" id="menu-item-9"><a href="https://data-x.blog/">About</a></li>
<li class="menu-item menu-item-type-post_type menu-item-object-page current-menu-item page_item page-item-94 current_page_item menu-item-102" id="menu-item-102"><a href="https://data-x.blog/syllabus-data-x/">Syllabus: Data-X</a></li>
<li class="menu-item menu-item-type-post_type menu-item-object-page menu-item-129" id="menu-item-129"><a href="https://data-x.blog/breakouts/">Breakouts</a></li>
<li class="menu-item menu-item-type-p

In [188]:
type(nav)

bs4.element.Tag

In [189]:
for url in nav.find_all('a'):
    print(url.get('href')) # only links in navigation bar

#content
/
https://data-x.blog/
https://data-x.blog/syllabus-data-x/
https://data-x.blog/breakouts/
https://data-x.blog/contact/


In [190]:
body = soup.body #get content within the <body> tag of the HTML code

In [191]:
# Print all body text
for paragraph in body.find_all('p'):
    print(paragraph.text) #might be two body tags. 
    # Just text from the body
    # Scraping for content

Instructor: Ikhlaq Sidhu
Department of Industrial Engineering & Operations Research
Offered Spring 2017, 3 Units, Lecture and Lab:
Prerequisite: Interested students should have working knowledge of Python in advance of the class, and also should have completed a fundamental probability or statistics course.
New Location: Now at 3108 Etcheverry Hall, Time: 5:10 pm-7:59
Not Barrows 60
Teaching Team:
Office Hours:
Tue 1:30-2:30pm at Etcheverry 4176 (Breakout Room B)
This course surveys a variety of key of concepts that are useful for designing and building applications that process data signals.  The course also introduces modern open source, computer programming tools and libraries that can be used to implement these applications.  These concepts include filtering, prediction, classification, decision-making, Markov chains, LTI systems, spectral analysis, and frameworks for learning from data.    After reviewing each concept, we explore implementing it within sample applications using Py

In [192]:
# Find all text within div sections, also child tags - or specific div section
for div in soup.find_all('div'):
    print(div.text) # a lot









Data-X
A Public and Open Website for the Data-X Course at UC Berkeley.


Menu
Skip to content
Home
About
Syllabus: Data-X
Breakouts
Contact
 






Syllabus: Data-X


Data-X: Data, Signals, and Systems
IEOR 190D/ 290-003
Spring 2017
Instructor: Ikhlaq Sidhu
Department of Industrial Engineering & Operations Research
Offered Spring 2017, 3 Units, Lecture and Lab:

Undergraduate Section: 190D, Class Number 33036
Graduate Section: Class Number INDENG 290 – 003, 33258

Prerequisite: Interested students should have working knowledge of Python in advance of the class, and also should have completed a fundamental probability or statistics course.
New Location: Now at 3108 Etcheverry Hall, Time: 5:10 pm-7:59
Not Barrows 60
Teaching Team:

GSI: Kevin Bozhe Li, kbl4ew@berkeley.edu
Visiting Scholar: Alexander Fred-Ojala, afo@berkeley.edu
Tensor Flow Lead: Nathan Cheng, ncheng@berkeley.edu
NLTK Lead: Sam Choi, sam.choi@berkeley.edu

Office Hours:
Tue 1:30-2:30pm at Etcheverry 4176 (Breakout

In [193]:
# prints both mobile and html version

for div in soup.find_all('div', class_='site-content'):
    print(div.text)






Syllabus: Data-X


Data-X: Data, Signals, and Systems
IEOR 190D/ 290-003
Spring 2017
Instructor: Ikhlaq Sidhu
Department of Industrial Engineering & Operations Research
Offered Spring 2017, 3 Units, Lecture and Lab:

Undergraduate Section: 190D, Class Number 33036
Graduate Section: Class Number INDENG 290 – 003, 33258

Prerequisite: Interested students should have working knowledge of Python in advance of the class, and also should have completed a fundamental probability or statistics course.
New Location: Now at 3108 Etcheverry Hall, Time: 5:10 pm-7:59
Not Barrows 60
Teaching Team:

GSI: Kevin Bozhe Li, kbl4ew@berkeley.edu
Visiting Scholar: Alexander Fred-Ojala, afo@berkeley.edu
Tensor Flow Lead: Nathan Cheng, ncheng@berkeley.edu
NLTK Lead: Sam Choi, sam.choi@berkeley.edu

Office Hours:
Tue 1:30-2:30pm at Etcheverry 4176 (Breakout Room B)
Description
This course surveys a variety of key of concepts that are useful for designing and building applications that process data signals

In [194]:
## Only get tables, scraping tables and xml documents

table = soup.table
table = soup.find('table')

In [195]:
table # shows the html code of the table

<table width="518">
<tbody>
<tr>
<td width="36"><strong>Lec #</strong></td>
<td width="131"><strong>Topic</strong></td>
<td width="72"><strong>Tools</strong></td>
<td width="108"><strong>Cookbook Examples</strong></td>
<td width="68"><strong>HW DUE</strong></td>
<td width="104"><strong>Lab<br/>
</strong></td>
</tr>
<tr>
<td width="36"><strong>1</strong>
<p>Jan 19</p></td>
<td width="131">Introduction: Overview of Frameworks for obtaining insights from data (Slides)
<p>Slides: Python and Math/Probability Pre-requisites</p></td>
<td width="72">Anaconda, Python</td>
<td width="108">Setting up Anaconda Environment</td>
<td width="68">HW 1 Assigned</td>
<td width="104"></td>
</tr>
<tr>
<td width="36"><strong>2</strong>
<p>Jan 26</p></td>
<td width="131">Notebook: Python Numpy Notebook
<p>Slides: Data Structure Outline</p>
<p>Slides: Numpy Review</p></td>
<td width="72">Python, Numpy, Pandas, JSON formatted files</td>
<td width="108">Earthquake Data live query
<p>Example with JSON file</p></

In [196]:
table_rows = table.find_all('tr') #table.tr or table.find('tr') would only find one

In [197]:
for tr in table_rows:
    td = tr.find_all('td') # find all table data
    row = [i.text for i in td]
    print(row) # get all the table data

['Lec #', 'Topic', 'Tools', 'Cookbook Examples', 'HW DUE', 'Lab\n']
['1\nJan 19', 'Introduction: Overview of Frameworks for obtaining insights from data (Slides)\nSlides: Python and Math/Probability Pre-requisites', 'Anaconda, Python', 'Setting up Anaconda Environment', 'HW 1 Assigned', '']
['2\nJan 26', 'Notebook: Python Numpy Notebook\nSlides: Data Structure Outline\nSlides: Numpy Review', 'Python, Numpy, Pandas, JSON formatted files', 'Earthquake Data live query\nExample with JSON file', 'Bring 3 ideas to next class\nHW 1 Due', 'Form Teams']
['3\nFeb 2', 'Data signals in Tables.\xa0 Slides: Pandas Overview\nNotebook: Pandas Intro\nNotebook: Pandas and Stock Market', 'Pandas, Numpy, SciPy, Matplotlib', 'Stock market live download to Pandas DataFrame. Quant trading algorithm', 'HW 2 Due', 'Form Teams']
['4\nFeb 9', 'Scoring, Linear Prediction and Max Likelihood Prediction. Extending to multiple variables', 'Numpy, SciPy, Matplotlib', 'Code samples: 2 variable and multi-variable Linear

In [198]:
# pandas version of grabbing tables, better

import pandas as pd

# requires html5lib: 
#!conda install --yes html5lib
dfs = pd.read_html('https://data-x.blog/syllabus-data-x/',header=0)
# header = 0, indicates that first row is header
# find all tables and parse them to several data frames



In [199]:
print(type(dfs))
print(len(dfs))
df = dfs[0]

<class 'list'>
1


In [200]:
# Looks great, but we might want the dates to be the indices and in datetimeformat
df.head()

,Lec #,Topic,Tools,Cookbook Examples,HW DUE,Lab
0,1 Jan 19,Introduction: Overview of Frameworks for obtai...,"Anaconda, Python",Setting up Anaconda Environment,HW 1 Assigned,NaN
1,2 Jan 26,Notebook: Python Numpy Notebook Slides: Data S...,"Python, Numpy, Pandas, JSON formatted files",Earthquake Data live query Example with JSON file,Bring 3 ideas to next class HW 1 Due,Form Teams
2,3 Feb 2,Data signals in Tables. Slides: Pandas Overvie...,"Pandas, Numpy, SciPy, Matplotlib",Stock market live download to Pandas DataFrame...,HW 2 Due,Form Teams
3,4 Feb 9,"Scoring, Linear Prediction and Max Likelihood ...","Numpy, SciPy, Matplotlib",Code samples: 2 variable and multi-variable Li...,HW 3 Due,Validate and Adjust
4,5 Feb 16,"Classification. Logistic Regression, SVM, Nonl...","Scikit Learn, Seaborn Visualization",Classification example with Iris Database: Log...,HW4 Due,Low Tech Demo and Validation Results


In [201]:
df.iloc[:,0] # dates are stored in the first column

0        1 Jan 19
1        2 Jan 26
2         3 Feb 2
3         4 Feb 9
4        5 Feb 16
5        6 Feb 23
6         7 Mar 2
7         8 Mar 9
8        9 Mar 16
9       10 Mar 23
10       11 Apr 6
11      12 Apr 13
12      12 Apr 20
13      13 Apr 27
14    Final May 4
Name: Lec #, dtype: object

In [202]:
dates = list() # list of better formatted dates
for date in df.iloc[:,0]:
    d = date.split()[1:] # split date strings and only extract Month plus Day, exclude lecture number
    d = '2017 ' + ' '.join(d)
    dates.append(d)

In [203]:
dates

['2017 Jan 19',
 '2017 Jan 26',
 '2017 Feb 2',
 '2017 Feb 9',
 '2017 Feb 16',
 '2017 Feb 23',
 '2017 Mar 2',
 '2017 Mar 9',
 '2017 Mar 16',
 '2017 Mar 23',
 '2017 Apr 6',
 '2017 Apr 13',
 '2017 Apr 20',
 '2017 Apr 27',
 '2017 May 4']

In [204]:
df.index=pd.to_datetime(dates) # convert dates to datetime objects and set them as the index
df.index.name='Date' #rename the index column to be "Date"

In [205]:
df.drop('Lec #',axis=1,inplace=True) # Drop the first column, with the old dates

In [226]:
df.head()

,Topic,Tools,Cookbook Examples,HW DUE,Lab
Date,,,,,
2017-01-19,Introduction: Overview of Frameworks for obtai...,"Anaconda, Python",Setting up Anaconda Environment,HW 1 Assigned,NaN
2017-01-26,Notebook: Python Numpy Notebook Slides: Data S...,"Python, Numpy, Pandas, JSON formatted files",Earthquake Data live query Example with JSON file,Bring 3 ideas to next class HW 1 Due,Form Teams
2017-02-02,Data signals in Tables. Slides: Pandas Overvie...,"Pandas, Numpy, SciPy, Matplotlib",Stock market live download to Pandas DataFrame...,HW 2 Due,Form Teams
2017-02-09,"Scoring, Linear Prediction and Max Likelihood ...","Numpy, SciPy, Matplotlib",Code samples: 2 variable and multi-variable Li...,HW 3 Due,Validate and Adjust
2017-02-16,"Classification. Logistic Regression, SVM, Nonl...","Scikit Learn, Seaborn Visualization",Classification example with Iris Database: Log...,HW4 Due,Low Tech Demo and Validation Results


In [227]:
pd.set_option('display.max_colwidth', -1) # to not get ... in the results

df.head()

,Topic,Tools,Cookbook Examples,HW DUE,Lab
Date,,,,,
2017-01-19,Introduction: Overview of Frameworks for obtaining insights from data (Slides) Slides: Python and Math/Probability Pre-requisites,"Anaconda, Python",Setting up Anaconda Environment,HW 1 Assigned,NaN
2017-01-26,Notebook: Python Numpy Notebook Slides: Data Structure Outline Slides: Numpy Review,"Python, Numpy, Pandas, JSON formatted files",Earthquake Data live query Example with JSON file,Bring 3 ideas to next class HW 1 Due,Form Teams
2017-02-02,Data signals in Tables. Slides: Pandas Overview Notebook: Pandas Intro Notebook: Pandas and Stock Market,"Pandas, Numpy, SciPy, Matplotlib",Stock market live download to Pandas DataFrame. Quant trading algorithm,HW 2 Due,Form Teams
2017-02-09,"Scoring, Linear Prediction and Max Likelihood Prediction. Extending to multiple variables","Numpy, SciPy, Matplotlib",Code samples: 2 variable and multi-variable Linear Prediction,HW 3 Due,Validate and Adjust
2017-02-16,"Classification. Logistic Regression, SVM, Nonlinear mapping","Scikit Learn, Seaborn Visualization","Classification example with Iris Database: Logistic, SVM",HW4 Due,Low Tech Demo and Validation Results


In [228]:
df.to_html('data-x-sched.html')

In [229]:
pd.options.display.max_colwidth=50 #change back to default max col_width

# Scrape images

In [257]:
print(soup.find_all('img'))

[<img alt="" height="154" src="https://datax911.files.wordpress.com/2016/12/cropped-banner_matrix1.jpg" width="912"/>, <img alt="course-model" class="alignnone size-full wp-image-99" data-attachment-id="99" data-comments-opened="1" data-image-description="" data-image-meta='{"aperture":"0","credit":"","camera":"","caption":"","created_timestamp":"0","copyright":"","focal_length":"0","iso":"0","shutter_speed":"0","title":"","orientation":"0"}' data-image-title="course-model" data-large-file="https://datax911.files.wordpress.com/2017/01/course-model.jpg?w=1032?w=911" data-medium-file="https://datax911.files.wordpress.com/2017/01/course-model.jpg?w=1032?w=300" data-orig-file="https://datax911.files.wordpress.com/2017/01/course-model.jpg?w=1032" data-orig-size="911,379" data-permalink="https://data-x.blog/syllabus-data-x/course-model/#main" sizes="(max-width: 911px) 100vw, 911px" src="https://datax911.files.wordpress.com/2017/01/course-model.jpg?w=1032" srcset="https://datax911.files.wordp

In [284]:
os.path.basename?

In [286]:
import os
import urllib

for link in soup.find_all('img'):
    img_url=link.get('src')
    
    if 'jpg' in img_url: #only check for jpg images
        print(img_url)
        print(os.path.splitext(os.path.basename(img_url))) # returns final component of pathname and extension as a tuple
        filename = os.path.splitext(os.path.basename(img_url))[0] + '.jpg'
        urllib.request.urlretrieve(img_url,filename) # urllib requests a file and then writes it to disk
    else:
        print('EXCLUDED:',img_url)

https://datax911.files.wordpress.com/2016/12/cropped-banner_matrix1.jpg
('cropped-banner_matrix1', '.jpg')
https://datax911.files.wordpress.com/2017/01/course-model.jpg?w=1032
('course-model', '.jpg?w=1032')
EXCLUDED: https://sb.scorecardresearch.com/p?c1=2&c2=7518284&c3=&c4=&c5=&c6=&c15=&cv=2.0&cj=1
EXCLUDED: https://pixel.wp.com/b.gif?v=noscript


In [ ]:
# XML documents - site maps, all the urls. just between tags
# XML human and machine readable.
# Newest links: all the links for FIND SITE MAP!
# News websites will have sitemaps for politics, bot constantly
# tracking news track the sitemaps

In [297]:
source = urllib.request.urlopen('https://data-x.blog/sitemap.xml').read()
soup = bs.BeautifulSoup(source,'xml') # interact with this object, looks like source in brower

In [299]:
soup.find_all('loc')

[<loc>https://data-x.blog/syllabus-data-x/</loc>,
 <image:loc>https://datax911.files.wordpress.com/2017/01/course-model.jpg</image:loc>,
 <loc>https://data-x.blog/breakouts/</loc>,
 <loc>https://data-x.blog/contact/</loc>,
 <image:loc>https://datax911.files.wordpress.com/2016/12/is-bio1.jpg</image:loc>,
 <loc>https://data-x.blog</loc>]

# Scrape Bloomberg for news

In [306]:
source = urllib.request.urlopen('https://www.bloomberg.com/feeds/bpol/sitemap_news.xml').read()
soup = bs.BeautifulSoup(source,'xml')

In [333]:
soup.prettify

<bound method Tag.prettify of <?xml version="1.0" encoding="utf-8"?>
<urlset xmlns="http://www.sitemaps.org/schemas/sitemap/0.9" xmlns:image="http://www.google.com/schemas/sitemap-image/1.1" xmlns:news="http://www.google.com/schemas/sitemap-news/0.9">
<url>
<loc>https://www.bloomberg.com/politics/articles/2017-03-07/fbi-s-comey-asked-to-testify-in-house-panel-s-russia-trump-probe</loc>
<news:news>
<news:publication>
<news:name>Bloomberg</news:name>
<news:language>en</news:language>
</news:publication>
<news:title>Comey Asked to Testify in House Panel's Russia-Trump Probe</news:title>
<news:publication_date>2017-03-08T00:01:34.173Z</news:publication_date>
<news:keywords>Cybersecurity, National Security, California, Russia, Richard Mauze Burr, Sally Yates, Adam B Schiff, Barack H Obama, Devin G Nunes, Donald John Trump, James Brien Comey</news:keywords>
<news:stock_tickers/>
</news:news>
<image:image>
<image:loc>https://assets.bwbx.io/images/users/iqjWHBFdfxIU/iSZah6P7k4uo/v0/1200x-1.jpg

In [346]:
for news in soup.find_all({'news'}):
    print(news.title.text)
    print(news.publication_date.text)
    print(news.keywords.text)
    print('\n')

Comey Asked to Testify in House Panel's Russia-Trump Probe
2017-03-08T00:01:34.173Z
Cybersecurity, National Security, California, Russia, Richard Mauze Burr, Sally Yates, Adam B Schiff, Barack H Obama, Devin G Nunes, Donald John Trump, James Brien Comey


Taxpayers May Be on the Hook for the Next SpaceX or Orbital Rocket Failure
2017-03-08T00:12:54.207Z
Congress, Engineering, Retirement, Transportation, Florida, Work, Washington, Science, White House, Cargo, Tech, NASA, Donald John Trump, Elon R Musk


Le Pen Promises to Resign If EU Exit Vote Fails, AFP Says
2017-03-07T16:31:41.051Z
France


House Speaker Paul Ryan Says GOP Health Bill Will Pass
2017-03-07T22:29:55.502Z
Law, Work, Washington, Health


Rep. Jordan Says Obamacare Repeal Plan Will Lower Costs
2017-03-07T22:22:37.303Z
Conservative, Jordan


London-Based Regulators in EU's Sights as Brexit Eviction Likely
2017-03-08T00:01:00.001Z
Luxembourg, Netherlands, Austria, France, Health, Banking, Sweden, Portugal, Ireland, U.K., Dr

In [ ]:
# example from https://www.ayima.com/guides/how-to-visualize-an-xml-sitemap-using-python.html

# Visualize XML sitemap with categories!
import requests
from bs4 import BeautifulSoup

url = 'https://www.sportchek.ca/sitemap.xml'
url = 'https://www.bloomberg.com/feeds/bpol/sitemap_index.xml'
page = requests.get(url)
print('Loaded page with: %s' % page)

sitemap_index = BeautifulSoup(page.content, 'html.parser')
print('Created %s object' % type(sitemap_index))

In [ ]:
urls = [element.text for element in sitemap_index.findAll('loc')]
print(urls)

In [ ]:
def extract_links(url):
    ''' Open an XML sitemap and find content wrapped in loc tags. '''

    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    links = [element.text for element in soup.findAll('loc')]

    return links

sitemap_urls = []
for url in urls:
    links = extract_links(url)
    sitemap_urls += links

print('Found {:,} URLs in the sitemap'.format(len(sitemap_urls)))

In [ ]:
with open('sitemap_urls.dat', 'w') as f:
    for url in sitemap_urls:
        f.write(url + '\n')

In [ ]:
'''
Categorize a list of URLs by site path.
The file containing the URLs should exist in the working directory and be
named sitemap_urls.dat. It should contain one URL per line.
Categorization depth can be specified by executing a call like this in the
terminal (where we set the granularity depth level to 5):
    python categorize_urls.py --depth 5
The same result can be achieved by setting the categorization_depth variable
manually at the head of this file and running the script with:
    python categorize_urls.py
'''
from __future__ import print_function


categorization_depth=3



# Main script functions


def peel_layers(urls, layers=3):
    ''' Builds a dataframe containing all unique page identifiers up
    to a specified depth and counts the number of sub-pages for each.
    Prints results to a CSV file.
    urls : list
        List of page URLs.
    layers : int
        Depth of automated URL search. Large values for this parameter
        may cause long runtimes depending on the number of URLs.
    '''

    # Store results in a dataframe
    sitemap_layers = pd.DataFrame()

    # Get base levels
    bases = pd.Series([url.split('//')[-1].split('/')[0] for url in urls])
    sitemap_layers[0] = bases

    # Get specified number of layers
    for layer in range(1, layers+1):

        page_layer = []
        for url, base in zip(urls, bases):
            try:
                page_layer.append(url.split(base)[-1].split('/')[layer])
            except:
                # There is nothing that deep!
                page_layer.append('')

        sitemap_layers[layer] = page_layer

    # Count and drop duplicate rows + sort
    sitemap_layers = sitemap_layers.groupby(list(range(0, layers+1)))[0].count()\
                     .rename('counts').reset_index()\
                     .sort_values('counts', ascending=False)\
                     .sort_values(list(range(0, layers)), ascending=True)\
                     .reset_index(drop=True)

    # Convert column names to string types and export
    sitemap_layers.columns = [str(col) for col in sitemap_layers.columns]
    sitemap_layers.to_csv('sitemap_layers.csv', index=False)

    # Return the dataframe
    return sitemap_layers




sitemap_urls = open('sitemap_urls.dat', 'r').read().splitlines()
print('Loaded {:,} URLs'.format(len(sitemap_urls)))

print('Categorizing up to a depth of %d' % categorization_depth)
sitemap_layers = peel_layers(urls=sitemap_urls,
                             layers=categorization_depth)
print('Printed {:,} rows of data to sitemap_layers.csv'.format(len(sitemap_layers)))


In [ ]:
'''
Visualize a list of URLs by site path.
This script reads in the sitemap_layers.csv file created by the
categorize_urls.py script and builds a graph visualization using Graphviz.
Graph depth can be specified by executing a call like this in the
terminal:
    python visualize_urls.py --depth 4 --limit 10 --title "My Sitemap" --style "dark" --size "40"
The same result can be achieved by setting the variables manually at the head
of this file and running the script with:
    python visualize_urls.py
'''
from __future__ import print_function


# Set global variables

graph_depth = 3  # Number of layers deep to plot categorization
limit = 3       # Maximum number of nodes for a branch
title = ''       # Graph title
style = 'light'  # Graph style, can be "light" or "dark"
size = '8,5'     # Size of rendered PDF graph


# Import external library dependencies

import pandas as pd
import graphviz



# Main script functions

def make_sitemap_graph(df, layers=3, limit=50, size='8,5'):
    ''' Make a sitemap graph up to a specified layer depth.
    sitemap_layers : DataFrame
        The dataframe created by the peel_layers function
        containing sitemap information.
    layers : int
        Maximum depth to plot.
    limit : int
        The maximum number node edge connections. Good to set this
        low for visualizing deep into site maps.
    '''


    # Check to make sure we are not trying to plot too many layers
    if layers > len(df) - 1:
        layers = len(df)-1
        print('There are only %d layers available to plot, setting layers=%d'
              % (layers, layers))


    # Initialize graph
    f = graphviz.Digraph('sitemap', filename='sitemap_graph_%d_layer' % layers)
    f.body.extend(['rankdir=LR', 'size="%s"' % size])


    def add_branch(f, names, vals, limit, connect_to=''):
        ''' Adds a set of nodes and edges to nodes on the previous layer. '''

        # Get the currently existing node names
        node_names = [item.split('"')[1] for item in f.body if 'label' in item]

        # Only add a new branch it it will connect to a previously created node
        if connect_to:
            if connect_to in node_names:
                for name, val in list(zip(names, vals))[:limit]:
                    f.node(name='%s-%s' % (connect_to, name), label=name)
                    f.edge(connect_to, '%s-%s' % (connect_to, name), label='{:,}'.format(val))


    f.attr('node', shape='rectangle') # Plot nodes as rectangles

    # Add the first layer of nodes
    for name, counts in df.groupby(['0'])['counts'].sum().reset_index()\
                          .sort_values(['counts'], ascending=False).values:
        f.node(name=name, label='{} ({:,})'.format(name, counts))

    if layers == 0:
        return f

    f.attr('node', shape='oval') # Plot nodes as ovals
    f.graph_attr.update()

    # Loop over each layer adding nodes and edges to prior nodes
    for i in range(1, layers+1):
        cols = [str(i_) for i_ in range(i)]
        nodes = df[cols].drop_duplicates().values
        for j, k in enumerate(nodes):

            # Compute the mask to select correct data
            mask = True
            for j_, ki in enumerate(k):
                mask &= df[str(j_)] == ki

            # Select the data then count branch size, sort, and truncate
            data = df[mask].groupby([str(i)])['counts'].sum()\
                    .reset_index().sort_values(['counts'], ascending=False)

            # Add to the graph
            add_branch(f,
                       names=data[str(i)].values,
                       vals=data['counts'].values,
                       limit=limit,
                       connect_to='-'.join(['%s']*i) % tuple(k))

            print(('Built graph up to node %d / %d in layer %d' % (j, len(nodes), i))\
                    .ljust(50), end='\r')

    return f


def apply_style(f, style, title=''):
    ''' Apply the style and add a title if desired. More styling options are
    documented here: http://www.graphviz.org/doc/info/attrs.html#d:style
    f : graphviz.dot.Digraph
        The graph object as created by graphviz.
    style : str
        Available styles: 'light', 'dark'
    title : str
        Optional title placed at the bottom of the graph.
    '''

    dark_style = {
        'graph': {
            'label': title,
            'bgcolor': '#3a3a3a',
            'fontname': 'Helvetica',
            'fontsize': '18',
            'fontcolor': 'white',
        },
        'nodes': {
            'style': 'filled',
            'color': 'white',
            'fillcolor': 'black',
            'fontname': 'Helvetica',
            'fontsize': '14',
            'fontcolor': 'white',
        },
        'edges': {
            'color': 'white',
            'arrowhead': 'open',
            'fontname': 'Helvetica',
            'fontsize': '12',
            'fontcolor': 'white',
        }
    }

    light_style = {
        'graph': {
            'label': title,
            'fontname': 'Helvetica',
            'fontsize': '18',
            'fontcolor': 'black',
        },
        'nodes': {
            'style': 'filled',
            'color': 'black',
            'fillcolor': '#dbdddd',
            'fontname': 'Helvetica',
            'fontsize': '14',
            'fontcolor': 'black',
        },
        'edges': {
            'color': 'black',
            'arrowhead': 'open',
            'fontname': 'Helvetica',
            'fontsize': '12',
            'fontcolor': 'black',
        }
    }

    if style == 'light':
        apply_style = light_style

    elif style == 'dark':
        apply_style = dark_style

    f.graph_attr = apply_style['graph']
    f.node_attr = apply_style['nodes']
    f.edge_attr = apply_style['edges']

    return f




# Read in categorized data
sitemap_layers = pd.read_csv('sitemap_layers.csv', dtype=str)
# Convert numerical column to integer
sitemap_layers.counts = sitemap_layers.counts.apply(int)
print('Loaded {:,} rows of categorized data from sitemap_layers.csv'\
        .format(len(sitemap_layers)))

print('Building %d layer deep sitemap graph' % graph_depth)
f = make_sitemap_graph(sitemap_layers, layers=graph_depth,
                       limit=limit, size=size)
f = apply_style(f, style=style, title=title)

f.render(cleanup=True)
print('Exported graph to sitemap_graph_%d_layer.pdf' % graph_depth)


